In [1]:
import numpy as np

import getfem as gf

initializing ...
numthread = 1


In [17]:
import pyvista as pv
from pyvirtualdisplay import Display

def render(edge = True):
    display = Display(visible=0, size=(1280, 1024))
    display.start()
    p = pv.Plotter()
    m = pv.read("feme.vtk")
    p.add_mesh(m, show_edges=edge, line_width=5)
    pts = m.points
    p.add_point_labels(pts, pts[:, 0].tolist(), point_size=10, font_size=10)
    p.show(window_size=[512, 384], jupyter_backend='panel', cpos="xy")
    display.stop()

In [18]:
center = [0.0, 0.0]
radius = 1.0
mo = gf.MesherObject("ball", center, radius)
mo

message from gf_mesher_object_get follow:
gfMesherObject object



In [22]:
h = 0.1
K = 2
mesh = gf.Mesh("generate", mo, h, K)
mesh.export_to_vtk("feme.vtk")
render()

effective bounding box : [-0.998954, -1] : [0.998954, 0.950032]
Iter 0 NEW DELAUNAY, running on 360 points
number of elements after selection = 672
Iter 5 NEW DELAUNAY, running on 360 points
number of elements after selection = 668
Iter 10 NEW DELAUNAY, running on 360 points
number of elements after selection = 668
Iter 15 NEW DELAUNAY, running on 360 points
number of elements after selection = 668
Iter 20 NEW DELAUNAY, running on 360 points
number of elements after selection = 666
Iter 25 NEW DELAUNAY, running on 360 points
number of elements after selection = 666
Iter 31 NEW DELAUNAY, running on 360 points
number of elements after selection = 664
Iter 38 NEW DELAUNAY, running on 360 points
number of elements after selection = 664
Iter 44 NEW DELAUNAY, running on 360 points
number of elements after selection = 660
Iter 52 NEW DELAUNAY, running on 360 points
number of elements after selection = 658
Iter 58 NEW DELAUNAY, running on 360 points
number of elements after selection = 658
Ite

In [34]:
outer_faces = mesh.outer_faces()
OUTER_BOUND = 1
mesh.set_region(OUTER_BOUND, outer_faces)

In [35]:
mf = gf.MeshFem(mesh, 1)
elements_degree = 2
mf.set_classical_fem(elements_degree)
mf

message from gf_mesh_fem_get follow:
gfMeshFem object in dimension 2 with 1375 points, 656 elements and 1375 degrees of freedom



In [36]:
mim = gf.MeshIm(mesh, pow(elements_degree, 2))
mim

message from gf_mesh_im_get follow:
gfMeshIm object in dimension 2 with 1375 points and 656 elements



In [37]:
md = gf.Model("real")
md

message from gf_model_get follow:
Real gfModel object with 0 degrees of freedom



In [38]:
md.add_fem_variable("u", mfu)
md.variable('u')

array([0., 0.])

In [39]:
md.add_Laplacian_brick(mim, "u")

0

In [40]:
F = 1.0
md.add_fem_data("F", mfu)


In [41]:
md.set_variable("F", np.repeat(F, mfu.nbdof()))


In [42]:
md.add_source_term_brick(mim, "u", "F")

1

In [43]:
md.add_Dirichlet_condition_with_multipliers(mim, "u", elements_degree - 1, OUTER_BOUND)

2

In [44]:
md.solve()

Trace 2 in getfem_models.cc, line 4386: Mass term assembly for Dirichlet condition
logic_error exception caught


RuntimeError: (Getfem::InterfaceError) -- Error in getfem_generic_assembly_compile_and_exec.cc, line 6109 void getfem::ga_compile_node(getfem::pga_tree_node, getfem::ga_workspace&, getfem::ga_instruction_set&, getfem::ga_instruction_set::region_mim_instructions&, const getfem::mesh&, bool, getfem::ga_if_hierarchy&): 
The finite element of variable u1u1 and the applied integration method have to be defined on the same mesh